In [55]:
import numpy as np
import pandas as pd

In [57]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [59]:
df = pd.read_csv("covid_toy.csv")
df.sample(5)

,age,gender,fever,cough,city,has_covid
91,38,Male,NaN,Mild,Delhi,Yes
18,64,Female,98.0,Mild,Bangalore,Yes
31,83,Male,103.0,Mild,Kolkata,No
83,17,Female,104.0,Mild,Kolkata,No
30,15,Male,101.0,Mild,Delhi,Yes


In [86]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [88]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = ["has_covid"]), df['has_covid'], test_size = 0.2, random_state = 0)
X_train.shape

(80, 5)

## Jobs to do:
- handle missing values from the fever column
- gender and city column -> one hot encoding
- cough -> ordinal encoding
- can do label encoding for the has_covid column

In [91]:
# Handling missing values
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape  # numpy ka array hai

(80, 1)

In [93]:
# Ordinal encoding -> cough
oe = OrdinalEncoder(categories = [["Mild", "Strong"]])

X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape # numpy ka array hai

(80, 1)

In [137]:
# OneHotEncoding -> gender and city
ohe = OneHotEncoder(drop='first')

X_train_gender = ohe.fit_transform(X_train[['gender']])
X_train_city = ohe.fit_transform(X_train[['city']])

X_test_gender = ohe.fit_transform(X_test[['gender']])
X_test_city = ohe.fit_transform(X_test[['city']])

city_columns = ohe.get_feature_names_out(['city'])
X_train_city_df = pd.DataFrame(X_train_city.toarray(), columns=city_columns, index=X_train.index)
X_train_city_df

,city_Delhi,city_Kolkata,city_Mumbai
43,0.0,0.0,0.0
62,0.0,0.0,0.0
3,0.0,1.0,0.0
71,1.0,0.0,0.0
45,0.0,0.0,0.0
...,...,...,...
96,0.0,1.0,0.0
67,0.0,0.0,0.0
64,0.0,0.0,1.0
47,0.0,0.0,0.0


In [97]:
# extracting the left alone age column
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [157]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender.toarray(),X_train_city_df.values,X_train_cough),axis=1)

#X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender,X_test_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

## Using Column transformers

In [151]:
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first'),['gender','city'])
],remainder='passthrough')

In [153]:
transformer.fit_transform(X_train).shape


(80, 7)

In [155]:
transformer.transform(X_test).shape

(20, 7)